In [1]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from keras.layers.merge import Concatenate
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import GRU, LSTM
from keras.layers import Conv1D, MaxPooling1D
import numpy as np
import pandas as pd
import random
from sklearn import tree
import copy
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.utils import resample
from keras.layers import Dropout, Dense, GRU, Embedding
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dropout, Dense,Input,Embedding,Flatten, AveragePooling2D, Conv2D,Reshape
from keras.models import Sequential,Model

Using TensorFlow backend.
/Users/ankursharma/anaconda3/envs/cv2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ankursharma/anaconda3/envs/cv2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ankursharma/anaconda3/envs/cv2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/anku

In [2]:
import pickle as pkl

In [3]:
# pro_file = '/Users/ankursharma/Desktop/MTP-org/dataset/all-policy/pro.txt'
# anti_file = '/Users/ankursharma/Desktop/MTP-org/dataset/all-policy/anti.txt'
# neutral
_file = '/Users/ankursharma/Desktop/MTP-org/dataset/all-policy/neutral.txt'

In [4]:
# policies = ['aadhar', 'gst', 'demon', 'farmers']
# policy = policies[1]
# print(policy)
# pro_file = '/Users/ankursharma/Desktop/MTP-org/dataset/relevant/' + policy + '/relevant.txt'
# anti_file = '/Users/ankursharma/Desktop/MTP-org/dataset/relevant/' + policy + '/non-relevant.txt'

In [5]:
# pro_file = '/Users/ankursharma/Desktop/MTP-org/dataset/final/econ_non_neutral.txt'
# anti_file = '/Users/ankursharma/Desktop/MTP-org/dataset/final/econ_neutral.txt'

pro_file = '/Users/ankursharma/Desktop/MTP-org/dataset/final/tech_non_neutral.txt'
anti_file = '/Users/ankursharma/Desktop/MTP-org/dataset/final/tech_neutral.txt'

# pro_file = '/Users/ankursharma/Desktop/MTP-org/dataset/final/processed/tech/pro.txt'
# anti_file = '/Users/ankursharma/Desktop/MTP-org/dataset/final/processed/tech/anti.txt'

# pro_file = '/Users/ankursharma/Desktop/MTP-org/gem2data/3train1testdataset/pro.txt'
# anti_file = '/Users/ankursharma/Desktop/MTP-org/gem2data/3train1testdataset/anti.txt'

# pro_file = '/Users/ankursharma/Desktop/MTP-org/dataset/final/processed/tech/relevant.txt'
# anti_file = '/Users/ankursharma/Desktop/MTP-org/dataset/final/processed/tech/non_relevant.txt'

In [6]:
def read_into_string(filename):
    text_file = open(filename, 'r')
    lines = text_file.read().split('\n')
    text_file.close()
    return lines

In [7]:
pro = read_into_string(pro_file) #1
anti = read_into_string(anti_file) #0
# neutral = read_into_string(neutral_file) #2

In [8]:
# len(pro), len(anti), len(neutral)
len(pro), len(anti)

(669, 135)

In [9]:
def split(dataset, label, train_test_split = 0.85):
    random.shuffle(dataset)
    idx = int(len(dataset) * train_test_split)
    train = dataset[: idx]
    test = dataset[idx + 1:]
    y_train = [label for x in range(len(train))]
    y_test = [label for x in range(len(test))]
    return (train, y_train), (test, y_test)

In [10]:
def balance_classes(X_train, y_train):
    dic = {}
    minm = 10000000
    for x, y in zip(X_train, y_train):
        if y in dic.keys():
            dic[y] = (1 + dic[y][0], [x] + dic[y][1])
        else:
            dic[y] = (1, [x])
    for k in dic.keys():
        if dic[k][0] < minm:
            minm = dic[k][0]
    X_train = []
    y_train = []
    for k in dic.keys():
        X_train += dic[k][1][: minm]
        y_train += [k for x in range(minm)]
    return shuffle(X_train, y_train)

In [11]:
def upsample(X_train, y_train):
    
    dic = {}
    minm = 10000000
    maxm = -minm
    X_final = []
    y_final = []
    
    # create the dictionary
    for x, y in zip(X_train, y_train):
        if y in dic.keys():
            dic[y] = (1 + dic[y][0], [x] + dic[y][1])
        else:
            dic[y] = (1, [x])
    
    # find maximum and minimum
    for k in dic.keys():
        count = dic[k][0]
        if dic[k][0] < minm:
            minm = count
        if dic[k][0] > maxm:
            maxm = count
            
    # Upsample all the non-majority classes
    for k in dic.keys():
        count = dic[k][0]
        examples = dic[k][1]
        if count < maxm:
            examples = resample(examples, 
                                          replace=True, # sample with replacement
                                          n_samples=maxm, # match number in majority class
                                          random_state=27) # reproducible results
            assert len(examples) == maxm
        X_final += examples
        y_final += [k for x in range(len(examples))]
        
    return shuffle(X_final, y_final)

In [12]:
(pro_X_train, pro_y_train), (pro_X_test, pro_y_test) = split(copy.deepcopy(pro), 1, train_test_split=0.85)
(anti_X_train, anti_y_train), (anti_X_test, anti_y_test) = split(copy.deepcopy(anti), 0, train_test_split=0.85)
# (neutral_X_train, neutral_y_train), (neutral_X_test, neutral_y_test) = split(copy.deepcopy(neutral), 2)

In [13]:
# pro v/s anti classification
X_train = pro_X_train + anti_X_train
y_train = pro_y_train + anti_y_train
X_test = pro_X_test + anti_X_test
y_test = pro_y_test + anti_y_test
X_train, y_train = shuffle(X_train, y_train)
X_test, y_test = shuffle(X_test, y_test)

In [14]:
# pro v/s anti v/s neutral classification
# X_train = pro_X_train + anti_X_train + neutral_X_train
# y_train = pro_y_train + anti_y_train + neutral_y_train
# X_test = pro_X_test + anti_X_test + neutral_X_test
# y_test = pro_y_test + anti_y_test + neutral_y_test
# X_train, y_train = shuffle(X_train, y_train)
# X_test, y_test = shuffle(X_test, y_test)

In [15]:
# # neutral v/s non-neutral classification
# X_train = pro_X_train + anti_X_train + neutral_X_train
# y_train = [1 for x in range(len(pro_y_train + anti_y_train))] + neutral_y_train
# X_test = pro_X_test + anti_X_test + neutral_X_test
# y_test = [1 for x in range(len(pro_y_test + anti_y_test))] + neutral_y_test
# X_train, y_train = shuffle(X_train, y_train)
# X_test, y_test = shuffle(X_test, y_test)
# # 1 for pro-anti, 2 for neutral
# mapping = {2: 0, 1: 1} # 1 for pro-anti, 0 for neutral
# y_train = [mapping[y] for y in y_train]
# y_test = [mapping[y] for y in y_test]

In [16]:
undersample = False

In [17]:
if undersample:
    X_train, y_train = balance_classes(X_train, y_train)
else:
    X_train, y_train = upsample(X_train, y_train)

In [18]:
len(y_train)

1136

### SVM

In [ ]:
def svm(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
svm(X_train, y_train, X_test, y_test)

### Rochhio Classification

In [ ]:
def rocchio(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', NearestCentroid()),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
rocchio(X_train, y_train, X_test, y_test)

### Random Forest Classifier

In [ ]:
def random_forest(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', RandomForestClassifier(n_estimators=100)),
                         ])

    text_clf.fit(X_train, y_train)
#     pkl.dump(text_clf, open('./relevance-tech-model.pkl', 'wb'))

    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
random_forest(X_train, y_train, X_test, y_test)

### RNN

In [19]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=50):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/Users/ankursharma/Desktop/MTP-org/dataset/glove.6B.50d.txt", encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [20]:
def Build_Model_RNN_Text(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=50, EMBEDDING_DIM=50, dropout=0.5):
    """
    def buildModel_RNN(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
    word_index in word index ,
    embeddings_index is embeddings index, look at data_helper.py
    nClasses is number of classes,
    MAX_SEQUENCE_LENGTH is maximum lenght of text sequences
    """

    model = Sequential()
    hidden_layer = 3
    gru_node = 256

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) != len(embedding_vector):
                print("could not broadcast input array from shape", str(len(embedding_matrix[i])),
                      "into shape", str(len(embedding_vector)), " Please make sure your"
                                                                " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))


    print(gru_node)
    for i in range(0,hidden_layer):
        model.add(GRU(gru_node,return_sequences=True, recurrent_dropout=0.2))
        model.add(Dropout(dropout))
    model.add(GRU(gru_node, recurrent_dropout=0.2))
   #model.add(Dense(, activation='relu'))
    model.add(Dense(nclasses, activation='softmax'))


    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model


In [21]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)


model_RNN = Build_Model_RNN_Text(word_index,embeddings_index, len(np.unique(X_test)))

model_RNN.summary()

Found 4161 unique tokens.
(1256, 50)
Total 400000 word vectors.
256
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            208100    
_________________________________________________________________
gru_1 (GRU)                  (None, 50, 256)           235776    
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 50, 256)           393984    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
gru_3 (GRU)                  (None, 50, 256)           393984    
____________________________________________________

In [22]:
model_RNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=30,
                              batch_size=32,
                              verbose=1,
                             shuffle=True)


Train on 1136 samples, validate on 120 samples
Epoch 1/30
1136/1136 [==============================] - 21s 18ms/step - loss: 7.0875 - accuracy: 0.4261 - val_loss: 2.8559 - val_accuracy: 0.1667
Epoch 2/30
1136/1136 [==============================] - 19s 17ms/step - loss: 1.4185 - accuracy: 0.5238 - val_loss: 2.2737 - val_accuracy: 0.1667
Epoch 3/30
1136/1136 [==============================] - 19s 17ms/step - loss: 1.0669 - accuracy: 0.5273 - val_loss: 1.2413 - val_accuracy: 0.2667
Epoch 4/30
1136/1136 [==============================] - 21s 18ms/step - loss: 0.7106 - accuracy: 0.5616 - val_loss: 1.2085 - val_accuracy: 0.6750
Epoch 5/30
1136/1136 [==============================] - 20s 18ms/step - loss: 0.6809 - accuracy: 0.5880 - val_loss: 1.0673 - val_accuracy: 0.6083
Epoch 6/30
1136/1136 [==============================] - 21s 19ms/step - loss: 0.6700 - accuracy: 0.6065 - val_loss: 1.0752 - val_accuracy: 0.7083
Epoch 7/30
1136/1136 [==============================] - 20s 18ms/step - loss

In [23]:
predicted = model_RNN.predict_classes(X_test_Glove)


print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.20      0.20      0.20        20
           1       0.84      0.84      0.84       100

    accuracy                           0.73       120
   macro avg       0.52      0.52      0.52       120
weighted avg       0.73      0.73      0.73       120



In [ ]:
sum(y_test)/len(y_test)

### RCNN

In [24]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=50):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/Users/ankursharma/Desktop/MTP-org/dataset/glove.6B.50d.txt", encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [25]:
def Build_Model_RCNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=50, EMBEDDING_DIM=50):

    kernel_size = 2
    filters = 256
    pool_size = 2
    gru_node = 256

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector



    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [26]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)


model_RCNN = Build_Model_RCNN_Text(word_index,embeddings_index, len(np.unique(y_test)))


model_RCNN.summary()

Found 4161 unique tokens.
(1256, 50)
Total 400000 word vectors.

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 50)            208100    
_________________________________________________________________
dropout_4 (Dropout)          (None, 50, 50)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 49, 256)           25856     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 24, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 23, 256)           131328    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 256)           0         
_______________________________________________________

In [28]:
model_RCNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=30,
                              batch_size=32,
                              verbose=1, shuffle=True)

predicted = model_RCNN.predict(X_test_Glove)

predicted = np.argmax(predicted, axis=1)
print(metrics.classification_report(y_test, predicted))

Train on 1136 samples, validate on 120 samples
Epoch 1/30
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0316 - accuracy: 0.9912 - val_loss: 1.2507 - val_accuracy: 0.7667
Epoch 2/30
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0387 - accuracy: 0.9903 - val_loss: 0.7583 - val_accuracy: 0.8083
Epoch 3/30
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0514 - accuracy: 0.9842 - val_loss: 1.2216 - val_accuracy: 0.6583
Epoch 4/30
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0275 - accuracy: 0.9903 - val_loss: 1.6152 - val_accuracy: 0.7167
Epoch 5/30
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0248 - accuracy: 0.9930 - val_loss: 1.4711 - val_accuracy: 0.7583
Epoch 6/30
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0205 - accuracy: 0.9956 - val_loss: 1.8158 - val_accuracy: 0.7417
Epoch 7/30
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0114 - accu

In [ ]:
predicted = model_RCNN.predict(X_test_Glove)
predicted = np.argmax(predicted, axis=1)
print(metrics.classification_report(y_test, predicted))
print(metrics.accuracy_score(y_test, predicted))

predicted = model_RCNN.predict(np.concatenate((X_train_Glove, X_test_Glove), axis=0))
predicted = np.argmax(predicted, axis=1)
print(metrics.classification_report(y_train + y_test, predicted))
metrics.accuracy_score(y_train + y_test, predicted)

In [ ]:
MAX_NB_WORDS=75000
MAX_SEQUENCE_LENGTH=50
text = np.concatenate((X_train, X_test), axis=0)
text = np.array(text)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text)

In [ ]:
import pickle 
pickle.dump((tokenizer, model_RCNN), open("/Users/ankursharma/Desktop/MTP-org/navreet-code/tech_classifier/models/rcnn_3_train_1_test.pkl", 'wb'))

In [ ]:
csv_file = '/Users/ankursharma/Desktop/MTP-org/gem2data/econ_mapping.csv'
df = pd.read_csv(csv_file,delimiter='#;;#')
df.head()

In [ ]:
MAX_NB_WORDS=75000
MAX_SEQUENCE_LENGTH=50
np.random.seed(7)
statements = list(df['By-Statement'])
np.random.seed(7)
text = np.concatenate((X_train, X_test), axis=0)
text = np.array(text)
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(statements)
statements = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Found %s unique tokens.' % len(word_index))
indices = np.arange(statements.shape[0])
# np.random.shuffle(indices)
statements = statements[indices]

In [ ]:
predicted = model_RCNN2.predict(statements)
predicted = np.argmax(predicted, axis=1)

In [ ]:
np.sum(predicted)/len(predicted)

In [ ]:
with open('/Users/ankursharma/Desktop/MTP-org/gem2data/econ_mapping_label.csv', 'w+') as fw:
    fw.write('Entity' + '#;;#' + 'Source' + '#;;#' + 'Predicted Label' + '#;;#' + 'By-Statement\n')
    for idx, line in enumerate(list(df['By-Statement'])):
        label = predicted[idx]
        entity = df['Entity'][idx]
        source = df['Source'][idx]
        if(label == 1):
            fw.write(entity + '#;;#' + source + '#;;#' + 'Pro' + '#;;#' + line + '\n')
        else:
            fw.write(entity + '#;;#' + source + '#;;#' + 'Anti' + '#;;#' + line + '\n')
    fw.close()

### Multinomial Naive Bayes

In [ ]:
def naive_bayes(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB()),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
naive_bayes(X_train, y_train, X_test, y_test)

### K Nearest Neighbors

In [ ]:
def knn(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', KNeighborsClassifier()),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
knn(X_train, y_train, X_test, y_test)

### Decision Trees

In [ ]:
def decision_trees(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', tree.DecisionTreeClassifier()),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
decision_trees(X_train, y_train, X_test, y_test)

### Deep Neural Networks

In [29]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

In [30]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [31]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)


model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], len(np.unique(X_test)))
model_DNN.summary()

tf-idf with 3751 features
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               1921024   
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)             

In [32]:
model_DNN.fit(X_train_tfidf, y_train,
                              validation_data=(X_test_tfidf, y_test),
                              epochs=200,
                              batch_size=16,
                              verbose=1,
                             shuffle=True)

Train on 1136 samples, validate on 120 samples
Epoch 1/200
1136/1136 [==============================] - 2s 2ms/step - loss: 1.3292 - accuracy: 0.4903 - val_loss: 0.6089 - val_accuracy: 0.8333
Epoch 2/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.3778 - accuracy: 0.8354 - val_loss: 0.7644 - val_accuracy: 0.8250
Epoch 3/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.1131 - accuracy: 0.9718 - val_loss: 0.8970 - val_accuracy: 0.8083
Epoch 4/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0678 - accuracy: 0.9815 - val_loss: 1.1522 - val_accuracy: 0.7917
Epoch 5/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0744 - accuracy: 0.9815 - val_loss: 1.1052 - val_accuracy: 0.7917
Epoch 6/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0656 - accuracy: 0.9815 - val_loss: 1.0961 - val_accuracy: 0.7917
Epoch 7/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0561

Epoch 57/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0246 - accuracy: 0.9912 - val_loss: 5.0089 - val_accuracy: 0.7917
Epoch 58/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0298 - accuracy: 0.9930 - val_loss: 4.3649 - val_accuracy: 0.8000
Epoch 59/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0240 - accuracy: 0.9912 - val_loss: 4.7252 - val_accuracy: 0.7917
Epoch 60/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0277 - accuracy: 0.9912 - val_loss: 3.6052 - val_accuracy: 0.7917
Epoch 61/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0277 - accuracy: 0.9921 - val_loss: 4.6474 - val_accuracy: 0.7917
Epoch 62/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0201 - accuracy: 0.9930 - val_loss: 4.7786 - val_accuracy: 0.7917
Epoch 63/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0308 - accuracy: 0.9903 - val_loss: 4.9699 -

1136/1136 [==============================] - 2s 2ms/step - loss: 0.0149 - accuracy: 0.9921 - val_loss: 6.3605 - val_accuracy: 0.7833
Epoch 114/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0143 - accuracy: 0.9938 - val_loss: 6.1861 - val_accuracy: 0.7750
Epoch 115/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0160 - accuracy: 0.9912 - val_loss: 6.6544 - val_accuracy: 0.7667
Epoch 116/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0166 - accuracy: 0.9921 - val_loss: 7.3467 - val_accuracy: 0.7833
Epoch 117/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0227 - accuracy: 0.9938 - val_loss: 5.4036 - val_accuracy: 0.7833
Epoch 118/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0156 - accuracy: 0.9912 - val_loss: 5.6152 - val_accuracy: 0.7833
Epoch 119/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0262 - accuracy: 0.9894 - val_loss: 4.3122 - val_ac

Epoch 169/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0110 - accuracy: 0.9938 - val_loss: 6.7153 - val_accuracy: 0.7583
Epoch 170/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0143 - accuracy: 0.9956 - val_loss: 6.7011 - val_accuracy: 0.7750
Epoch 171/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0097 - accuracy: 0.9947 - val_loss: 6.7380 - val_accuracy: 0.7750
Epoch 172/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0077 - accuracy: 0.9965 - val_loss: 8.1315 - val_accuracy: 0.7833
Epoch 173/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0045 - accuracy: 0.9974 - val_loss: 8.7437 - val_accuracy: 0.7833
Epoch 174/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0131 - accuracy: 0.9930 - val_loss: 9.6088 - val_accuracy: 0.7500
Epoch 175/200
1136/1136 [==============================] - 2s 2ms/step - loss: 0.0182 - accuracy: 0.9912 - val_loss: 7

In [33]:
predicted = model_DNN.predict_classes(X_test_tfidf)

print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.25      0.15      0.19        20
           1       0.84      0.91      0.87       100

    accuracy                           0.78       120
   macro avg       0.55      0.53      0.53       120
weighted avg       0.74      0.78      0.76       120



### CNN

In [34]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=50):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/Users/ankursharma/Desktop/MTP-org/dataset/glove.6B.100d.txt", encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [35]:
def Build_Model_CNN_Text(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=50, EMBEDDING_DIM=100, dropout=0.5):

    """
        def buildModel_CNN(word_index, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=100, dropout=0.5):
        word_index in word index ,
        embeddings_index is embeddings index, look at data_helper.py
        nClasses is number of classes,
        MAX_SEQUENCE_LENGTH is maximum lenght of text sequences,
        EMBEDDING_DIM is an int value for dimention of word embedding look at data_helper.py
    """

    model = Sequential()
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("could not broadcast input array from shape",str(len(embedding_matrix[i])),
                                 "into shape",str(len(embedding_vector))," Please make sure your"
                                 " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True)

    # applying a more complex convolutional approach
    convs = []
    filter_sizes = []
    layer = 5
    print("Filter  ",layer)
    for fl in range(0,layer):
        filter_sizes.append((fl+2,fl+2))

    node = 128
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    emb = Reshape((MAX_SEQUENCE_LENGTH,10, 10), input_shape=(MAX_SEQUENCE_LENGTH,100))(embedded_sequences)

    for fsz in filter_sizes:
        l_conv = Conv2D(node, padding="same", kernel_size=fsz, activation='relu')(emb)
        l_pool = AveragePooling2D(pool_size=(5,1), padding="same")(l_conv)
        #l_pool = Dropout(0.25)(l_pool)
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)
    l_cov1 = Conv2D(node, (5,5), padding="same", activation='relu')(l_merge)
    l_cov1 = AveragePooling2D(pool_size=(5,2), padding="same")(l_cov1)
    l_cov2 = Conv2D(node, (5,5), padding="same", activation='relu')(l_cov1)
    l_pool2 = AveragePooling2D(pool_size=(5,2), padding="same")(l_cov2)
    l_cov2 = Dropout(dropout)(l_pool2)
    l_flat = Flatten()(l_cov2)
    l_dense = Dense(128, activation='relu')(l_flat)
    l_dense = Dropout(dropout)(l_dense)

    preds = Dense(nclasses, activation='softmax')(l_dense)
    model = Model(sequence_input, preds)

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])



    return model

In [36]:
X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)


model_CNN = Build_Model_CNN_Text(word_index,embeddings_index, len(np.unique(X_test)))


model_CNN.summary()

Found 4161 unique tokens.
(1256, 50)
Total 400000 word vectors.
Filter   5

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 50, 100)      416200      input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 50, 10, 10)   0           embedding_3[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 50, 10, 128)  5248        reshape_1[0][0]                  
________________

In [37]:
model_CNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=10,
                              batch_size=32,
                              verbose=1,
                                shuffle=True)

Train on 1136 samples, validate on 120 samples
Epoch 1/10
1136/1136 [==============================] - 18s 15ms/step - loss: 1.4487 - accuracy: 0.4674 - val_loss: 0.6219 - val_accuracy: 0.8333
Epoch 2/10
1136/1136 [==============================] - 17s 15ms/step - loss: 0.7749 - accuracy: 0.5176 - val_loss: 0.7004 - val_accuracy: 0.4000
Epoch 3/10
1136/1136 [==============================] - 17s 15ms/step - loss: 0.7228 - accuracy: 0.5458 - val_loss: 0.7090 - val_accuracy: 0.3833
Epoch 4/10
1136/1136 [==============================] - 17s 15ms/step - loss: 0.7145 - accuracy: 0.5352 - val_loss: 0.7283 - val_accuracy: 0.3917
Epoch 5/10
1136/1136 [==============================] - 18s 16ms/step - loss: 0.6650 - accuracy: 0.5924 - val_loss: 0.5682 - val_accuracy: 0.6667
Epoch 6/10
1136/1136 [==============================] - 17s 15ms/step - loss: 0.4212 - accuracy: 0.8178 - val_loss: 0.4479 - val_accuracy: 0.8083
Epoch 7/10
1136/1136 [==============================] - 17s 15ms/step - loss:

In [39]:
predicted = model_CNN.predict(X_test_Glove)

predicted = np.argmax(predicted, axis=1)


print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.27      0.30      0.29        20
           1       0.86      0.84      0.85       100

    accuracy                           0.75       120
   macro avg       0.56      0.57      0.57       120
weighted avg       0.76      0.75      0.75       120



### Gradient Boosting Classifier

In [ ]:
def gradient_boosting(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', GradientBoostingClassifier(n_estimators=50,verbose=2)),
                         ])

    
    text_clf.fit(X_train, y_train)
#     pkl.dump(text_clf, open('./' + policy + '-model.pkl', 'wb'))
    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
gradient_boosting(X_train, y_train, X_test, y_test)

### Bag of Words

In [ ]:
def bow(X_train, y_train, X_test, y_test):
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', BaggingClassifier(KNeighborsClassifier())),
                         ])

    text_clf.fit(X_train, y_train)


    predicted = text_clf.predict(X_test)

    print(metrics.classification_report(y_test, predicted))

In [ ]:
bow(X_train, y_train, X_test, y_test)